<a href="https://colab.research.google.com/github/vasudevarao3/DIGIBHEM/blob/main/Copy_of_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
!pip install yfinance

In [ ]:
!pip install pandas_market_calendars

In [ ]:
!pip install scikeras

In [ ]:
!pip install --upgrade fastapi pydantic

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as datetime
import pandas_market_calendars as pmcal
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Attention, Input
from tensorflow.keras.models import Model
from scikeras.wrappers import KerasRegressor

In [ ]:
# ticker = input("Enter the strock symbol:")
stock_ticker = yf.Ticker("SBUX")
start_date = datetime.date(2018,1,1)
end_date = datetime.date.today()
data = stock_ticker.history(start = start_date, end = end_date)
data

In [ ]:
data.index = data.index.date

In [ ]:
data = data.reset_index(drop = False)
data.rename(columns={'index': 'Dates'}, inplace=True)  # Optional: rename column to "row_index"
type(pd.to_datetime(data["Dates"]))
data

In [ ]:
data["Dates"] = pd.to_datetime(data["Dates"])
data.set_index(data["Dates"], inplace = True)  # Set the copied column as the index
data

In [ ]:
data.head()

#Task - 1

In [ ]:
data["Close"]

In [ ]:
scaler = MinMaxScaler(feature_range = (0,1))
data["Scaled_close"] = scaler.fit_transform(pd.DataFrame(data["Close"]))
data

In [ ]:
train_start_date = datetime.date(2018, 1, 1)
train_end_date = datetime.date(2022, 12, 31)
print(type(train_start_date))

main_train_data = data.loc[train_start_date:train_end_date]
print(main_train_data.shape)
main_train_data

In [ ]:
test_start_date = datetime.date(2023, 1, 1)
test_end_date = datetime.date(2023, 12, 31)

main_test_data = data.loc[test_start_date:test_end_date]
print(main_test_data.shape)
main_test_data

In [ ]:
nse = pmcal.get_calendar('NSE')

forecast_start_date = datetime.date(2024, 1, 1)
forecast_end_date = datetime.date(2024, 12, 31)
forecast_dates = nse.schedule(start_date= forecast_start_date, end_date = forecast_end_date)
print(forecast_dates)
forecast_dates = forecast_dates.index
forecast_dates

#Task - 2

In [ ]:
eval = {}
def evaluate_models(model, name, X_train, y_train, X_test, y_test):
  if(name not in eval.keys()):
    eval[name] = {}
    eval[name]["Mean Absolute Error"] ={}
    eval[name]["Mean Absolute Error"]["train data"] = 0
    eval[name]["Mean Absolute Error"]["test data"] = 0
    eval[name]["Root Mean Squared Error"] ={}
    eval[name]["Root Mean Squared Error"]["train data"] = 0
    eval[name]["Root Mean Squared Error"]["test data"] = 0
    eval[name]["R-Squared Score"] ={}
    eval[name]["R-Squared Score"]["train data"] = 0
    eval[name]["R-Squared Score"]["test data"] = 0

  print("="*25, name, "="*25)
  train_pred = model.predict(X_train)
  test_pred = model.predict(X_test)

  if(name == "Hybrid Model" or  name == "Attention Model"):
    train_pred = train_pred[:, -1, 0]
    test_pred = test_pred[:, -1, 0]

  print(train_pred.shape, y_train.shape, X_train.shape)
  print(test_pred.shape, y_test.shape, X_test.shape)

  y_train = scaler.inverse_transform(y_train.reshape(-1, 1))
  train_pred = scaler.inverse_transform(train_pred.reshape(-1, 1))
  y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
  test_pred = scaler.inverse_transform(test_pred.reshape(-1, 1))

  #Mean Absolute Error
  train_mae = mean_absolute_error(train_pred, y_train)
  test_mae = mean_absolute_error(test_pred, y_test)
  print("Mean Absolute Errors is :\n \t\t trained data: ",train_mae,"\n \t\t test data: ", test_mae)
  eval[name]["Mean Absolute Error"]["train data"] = train_mae
  eval[name]["Mean Absolute Error"]["test data"] = test_mae
  print("-"*50)

  #Root Mean Square Error
  train_rmse = root_mean_squared_error(train_pred, y_train)
  test_rmse = root_mean_squared_error(test_pred, y_test)
  print("Root Mean squared Error is :\n \t\t trained data: ",train_rmse,"\n \t\t test data: ", test_rmse)
  eval[name]["Root Mean Squared Error"]["train data"] = train_rmse
  eval[name]["Root Mean Squared Error"]["test data"] = test_rmse
  print("-"*50)

  #R-Squared Error
  train_rs = r2_score(train_pred, y_train)
  test_rs = r2_score(test_pred, y_test)
  print("R-Squared score is :\n \t\t trained data: ",train_rs,"\n \t\t test data: ", test_rs)
  eval[name]["R-Squared Score"]["train data"] = train_rs
  eval[name]["R-Squared Score"]["test data"] = test_rs

  # plotting(model, name, X_train, y_train, X_test, y_test, train_pred, test_pred)
  plt.figure(figsize = (20,5))
  plt.plot(main_train_data.index[no_of_days_to_analyse:], y_train, color = "red", label = "Actual")
  plt.plot(main_train_data.index[no_of_days_to_analyse:], train_pred, color = "blue", label = "Predicted")
  plt.title("Train Data v/s Predicted Train data")
  plt.legend()
  plt.show()

  print()
  plt.figure(figsize = (20,5))
  plt.plot(main_test_data.index[no_of_days_to_analyse:], y_test, color = "orange", label = "Actual")
  plt.plot(main_test_data.index[no_of_days_to_analyse:], test_pred, color = "yellow", label = "Predicted")
  plt.title("Test Data v/s Predicted Test data")
  plt.legend()



In [ ]:
#Creating dataset with definite number of columns with closing price to predict next day closing price

# no_of_days_to_analyse = int(input("Enter number of days to analyse:"))
no_of_days_to_analyse = 60
def create_dataset(data, day_step):
    X = []
    y = []
    for i in range(day_step, len(data)):
        X.append(data[i - day_step:i])
        y.append(data[i])
    return np.array(X), np.array(y)

In [ ]:
X_train_data, Y_train_data = create_dataset(main_train_data["Scaled_close"], no_of_days_to_analyse)
X_test_data, Y_test_data = create_dataset(main_test_data["Scaled_close"], no_of_days_to_analyse)

In [ ]:
pd.DataFrame(X_train_data)

In [ ]:
pd.DataFrame(Y_train_data)

In [ ]:
main_train_data.index[no_of_days_to_analyse:]

In [ ]:
pd.DataFrame(X_test_data)

In [ ]:
pd.DataFrame(Y_test_data)

In [ ]:
main_test_data.index[no_of_days_to_analyse:]

In [ ]:
X_train_data.shape

In [ ]:
Y_train_data.shape

In [ ]:
X_test_data.shape

In [ ]:
Y_test_data.shape

In [ ]:
X_train_data = X_train_data.reshape(X_train_data.shape[0], X_train_data.shape[1], 1)
X_test_data = X_test_data.reshape(X_test_data.shape[0], X_test_data.shape[1], 1)

In [ ]:
X_test_data.shape

In [ ]:
Y_test_data.shape

In [ ]:
X_train_data.shape

###LSTM

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train_data.shape[1], 1)))
lstm_model.add(LSTM(units = 50, return_sequences = True))
lstm_model.add(LSTM(units = 1))

In [ ]:
lstm_model.compile(loss = "mean_squared_error", optimizer = "adam")

In [ ]:
lstm_model.summary()

In [ ]:
lstm_model.fit(X_train_data, Y_train_data, validation_data = (X_test_data, Y_test_data), epochs = 60, batch_size = 50, verbose = 1)

In [ ]:
#LSTM
evaluate_models(lstm_model, "LSTM(Long Short Term Memory)", X_train_data, Y_train_data, X_test_data, Y_test_data)

###GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(units = 50, return_sequences = True, input_shape = (X_train_data.shape[1], 1)))
gru_model.add(GRU(units = 50, return_sequences = True))
gru_model.add(GRU(units = 1))

In [ ]:
gru_model.compile(loss = "mean_squared_error", optimizer = "adam")

In [ ]:
gru_model.summary()

In [ ]:
gru_model.fit(X_train_data, Y_train_data, validation_data = (X_test_data, Y_test_data), epochs = 60, batch_size = 50, verbose = 1)

In [ ]:
evaluate_models(gru_model, "GRU(Gated Recurrent Unit)", X_train_data, Y_train_data, X_test_data, Y_test_data)

###Hybrid Model(LSTM + Dense Layer)

In [ ]:
X_train_data.shape

In [ ]:
hybrid_model = Sequential()
hybrid_model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train_data.shape[1], 1)))
hybrid_model.add(LSTM(units = 50, return_sequences = True))
hybrid_model.add(LSTM(units = 50, return_sequences = True))

hybrid_model.add(Dense(units=25, activation='relu'))
hybrid_model.add(Dense(units=10, activation='relu'))
hybrid_model.add(Dense(units=1))

In [ ]:
hybrid_model.compile(loss = "mean_squared_error", optimizer = "adam")

In [ ]:
hybrid_model.summary()

In [ ]:
hybrid_model.fit(X_train_data, Y_train_data, validation_data = (X_test_data, Y_test_data), epochs = 60, batch_size = 50, verbose = 1)

In [ ]:
print(X_train_data.shape, X_test_data.shape, Y_train_data.shape, Y_test_data.shape)

In [ ]:
evaluate_models(hybrid_model, "Hybrid Model", X_train_data, Y_train_data, X_test_data, Y_test_data)

###Attention Model(LSTM + Attention)

In [ ]:
print(X_train_data.shape, X_test_data.shape, Y_train_data.shape, Y_test_data.shape)

In [ ]:
inputs = Input(shape=(X_train_data.shape[1], 1))
print(inputs)
x = LSTM(units=50, return_sequences=True)(inputs)
x = LSTM(units=50, return_sequences=True)(x)
attention = Attention()([x, x])  # Attention with query and key as x
print(attention)
outputs = Dense(1)(attention)
print(outputs)

attention_model = Model(inputs, outputs)
attention_model

In [ ]:
attention_model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
attention_model.summary()

In [ ]:
attention_model.fit(X_train_data, Y_train_data, validation_data = (X_test_data, Y_test_data), epochs = 60, batch_size = 50, verbose = 1)

In [ ]:
evaluate_models(attention_model, "Attention Model", X_train_data, Y_train_data, X_test_data, Y_test_data)

##Forecasting

In [ ]:
for mdl in eval.keys():
  print("-"*25,mdl,"-"*25)
  df = pd.DataFrame(eval[mdl])
  print(df)

In [ ]:
forecast_dates

In [ ]:
Y_forecast_prev_data = Y_test_data[-no_of_days_to_analyse:]
Y_forecast_prev_data.shape

In [ ]:
def forecast_dataset(model, name, data, column_size, forecast_dates):
  y = []
  data = data.reshape(1, data.shape[0], 1)
  # print(model, data, column_size, forecast_dates)
  for i in range(len(forecast_dates)):
    new_value = model.predict(data[:, i:i + column_size, :])
    data = np.append(data, new_value.reshape(1, 1, new_value.shape[1]), axis=1)
    new_value = scaler.inverse_transform(new_value)
    y.append(new_value)
  return np.array(y)


def forecast(model, name, Y_forecast_data):
  Y_forecast_data.reshape(Y_forecast_data.shape[0])
  date_filtered = data[data.index >= '2024-01-01']

  plt.figure(figsize = (18,5))
  plt.plot(forecast_dates, Y_forecast_data.reshape(Y_forecast_data.shape[0], 1), color = "orange", label = "Actual")
  plt.plot(date_filtered.index, date_filtered["Close"], color = "yellow", label = "Predicted")
  plt.title(f"Actual Data v/s Predicted Test data for {name}")
  plt.legend()
  plt.show()
  print()


In [ ]:
Y_lstm_forecast_data = forecast_dataset(lstm_model, "LSTM", Y_forecast_prev_data, no_of_days_to_analyse, forecast_dates)

Y_gru_forecast_data = forecast_dataset(gru_model, "GRU", Y_forecast_prev_data, no_of_days_to_analyse, forecast_dates)

In [ ]:
forecast(lstm_model, "LSTM(Long Short Term Memory)", Y_lstm_forecast_data)

forecast(gru_model, "GRU(Gated Recurrent Unit)", Y_gru_forecast_data)

#Hyperparameter Tuning

In [ ]:
units = [20, 30, 40, 50, 60, 70, 80, 90]
learning_rate = [0.001, 0.003, 0.005, 0.007, 0.009, 0.01]
batch_size = [30, 50, 70, 90, 100]
epochs = [60, 80, 100, 120]
lstm_params_grid = {
    'batch_size': batch_size,
    'epochs': epochs
}

gru_params_grid = {
    'batch_size': batch_size,
    'epochs': epochs
}

#####LSTM

In [ ]:
lstm_model_wrapped = KerasRegressor(model=lstm_model, metrics=['accuracy'], verbose=0)
lstm_model_grid = GridSearchCV(estimator=lstm_model_wrapped, param_grid=lstm_params_grid, cv=3, n_jobs=-1, scoring="accuracy")
lstm_model_grid

In [ ]:
lstm_model_grid.get_params().keys()

In [120]:
lstm_model_grid.fit(X_train_data, Y_train_data)
evaluate_models(lstm_model_grid, "LSTM(Long Short Term Memory) After Grid Search Tuning", X_train_data, Y_train_data, X_test_data, Y_test_data)

KeyboardInterrupt: 

In [ ]:
lstm_model_wrapped = KerasRegressor(model=lstm_model, metrics=['accuracy'], verbose=0)
lstm_randomized_search = RandomizedSearchCV(estimator=lstm_model_wrapped, param_distributions=lstm_params_grid, cv=3, n_iter=10, n_jobs=-1)
lstm_randomized_search

In [ ]:
lstm_randomized_search.get_params().keys()

In [ ]:
lstm_randomized_search.estimator

In [ ]:
lstm_randomized_search.fit(X_train_data, Y_train_data)
evaluate_models(lstm_randomized_search, "LSTM(Long Short Term Memory) After Randomized Search Turing", X_train_data, Y_train_data, X_test_data, Y_test_data)

#####GRU

In [ ]:
gru_model_wrapped = KerasRegressor(model=gru_model, metrics=['accuracy'], verbose=0)
gru_model_grid = GridSearchCV(estimator=gru_model_wrapped, param_grid=gru_params_grid, cv=3, n_jobs=-1, scoring="accuracy")
gru_model_grid

In [ ]:
gru_model_grid.get_params().keys()

In [ ]:
gru_model_grid.fit(X_train_data, Y_train_data)
evaluate_models(gru_model_grid, "GRU(Gated Recurrent Unit) After Grid Search Tuning", X_train_data, Y_train_data, X_test_data, Y_test_data)

In [ ]:
gru_model_wrapped = KerasRegressor(model=gru_model, metrics=['accuracy'], verbose=0)
gru_randomized_search = RandomizedSearchCV(estimator=gru_model_wrapped, param_distributions=lstm_params_grid, cv=3, n_iter=10, n_jobs=-1)
gru_randomized_search

In [ ]:
gru_randomized_search.get_params().keys()

In [ ]:
gru_randomized_search.fit(X_train_data, Y_train_data)
evaluate_models(gru_randomized_search, "GRU(Gated Recurrent Unit) After Randomized Search Turing", X_train_data, Y_train_data, X_test_data, Y_test_data)